In [1]:
import os
import numpy as np
import pandas as pd
from utils.gridsearch import gridsearch
from utils.read2df import read2df
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint

import backtrader as bt
from itertools import combinations

In [2]:
symbols = ['BTCUSDT', 'ETHUSDT']
start_date = '2023-01-01'

# freqs = {'1m': 1, '3m': 3, '5m': 5, '15m': 15, '30m': 30, '1h':60, '2h': 120}
freqs = {'1d': 1440}

# Download Data from [binance-public-data](https://github.com/binance/binance-public-data/tree/master/python)

Download BTCUSDT and ETHUSDT for all available history for intervals of 1m, 3m, 5m, 15m, 30m


In [ ]:
%%capture
if symbols is None:
    !python binance-public-data/python/download-kline.py -i {" ".join(list(freqs.keys()))} -startDate {start_date} -t spot -skip-daily 1
else:
    !python binance-public-data/python/download-kline.py -s {" ".join(symbols)} -i {" ".join(list(freqs.keys()))} -startDate {start_date} -t spot -skip-daily 1

In [3]:
# dfs = read2df(symbols, freqs)
dfs = read2df(symbols, freqs)

# Check Cointegration and Correlation

In [6]:
for i, (FREQUENCY, f) in enumerate(freqs.items()):
    print(f"Under {FREQUENCY} interval")
    for comb in combinations(set(dfs[i]['tic']), 2):
        first_ele = dfs[i][dfs[i]['tic'] == comb[0]]['close']
        second_ele = dfs[i][dfs[i]['tic'] == comb[1]]['close']
        _, pvalue, _ = coint(first_ele, second_ele)
        if pvalue <= 0.05:
            print(f"{comb[0]} and {comb[1]} are cointegrated, with correlation as {np.corrcoef(first_ele, second_ele)[0][1]}")
        else:
            print(f"{comb[0]} and {comb[1]} are NOT cointegrated under {FREQUENCY} interval")

Under 1d interval
BTCUSDT and ETHUSDT are NOT cointegrated under 1d interval


# Define Trading Strategy

Firstly define a sizer based on [Kelly Criterion](https://www.wikiwand.com/en/Kelly_criterion)

In [170]:
# Seems that Sizer can only be executed when self.buy(size=None). 
# We need to purchase amount in a certain ratio in Pair Trading.
# Therefore the Sizer is hard to implemented.

class KellyCriterionSizer(bt.Sizer):
    params = (('period', 30),)

    def _getsizing(self, comminfo, cash, data, isbuy):
        position = self.broker.getposition(data).size

        close_prices = data.close.get(size=self.p.period)
        returns = np.log(close_prices / close_prices.shift(1)).dropna()

        p = len(returns[returns > 0]) / len(returns)
        a = (returns[returns > 0].mean() + 1) if len(returns[returns > 0]) > 0 else 1.0
        b = (-returns[returns > 0].mean() + 1) if len(returns[returns < 0]) > 0 else 1.0
        q = 1 - p

        f = min(max((p/a - q/b), 0), 1)

        if isbuy:
            size = cash * f / data.close[0]
        else:
            size = position * f

        return size

Define a custom indicator for [Kelly Criterion](https://www.wikiwand.com/en/Kelly_criterion)

In [200]:
class KellyCriterionIndicator(bt.indicators.PeriodN):
    '''
    Uses ``pandas``
    '''
    _mindatas = 1

    packages = (
        ('pandas', 'pd'),
    )
    lines = ('kc_f',)
    params = (
        ('period', 30),
    )

    def next(self):
        spreads = pd.Series(self.data.get(size=self.p.period))
        returns = spreads.pct_change()

        kc_p = len(returns[returns > 0]) / len(returns)
        kc_a = (returns[returns > 0].mean() + 1) if len(returns[returns > 0]) > 0 else 1
        kc_b = (returns[returns < 0].mean() + 1) if len(returns[returns < 0]) > 0 else 1
        kc_q = 1 - kc_p
        
        kc_f = min(max((kc_p/kc_a - kc_q/kc_b), 0), 1)
        self.lines.kc_f[0] = kc_f

Define custom CommissionInfo

In [218]:
class PairTradingCommInfo(bt.CommInfoBase):
    params = (
        ('commission', 0.0), ('mult', 10), ('margin', 1000),
        ('stocklike', False),
        ('commtype', bt.CommInfoBase.COMM_PERC),
        ('percabs', True),
    )

The strategy with fixed ordersize

In [211]:
class PairTrading(bt.Strategy):
    params = dict(
        OPEN_THRE=2,
        CLOS_THRE=0.1,
        period=30
    )

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status == order.Completed:
            if order.isbuy():
                print(f"Buy {order.data._name} @ price: {order.executed.price} for Qty: {order.executed.size}")
            else:
                print(f"Sell {order.data._name} @ price: {order.executed.price} for Qty: {order.executed.size}")

        elif order.status in [order.Expired, order.Canceled, order.Margin]:
            print('%s ,' % order.Status[order.status])
            pass

    def __init__(self):
        self.data0 = self.datas[0]
        self.data1 = self.datas[1]

        # Calculate zscore of the ratio
        transform = bt.indicators.OLS_TransformationN(self.data1, self.data0, period=self.p.period)
        spread = transform.spread
        self.zscore = transform.zscore

        self.kc_f = KellyCriterionIndicator(spread, period=self.p.period)

        # -1 for short data1/data0, 1 for long data1/data0, 0 for no position
        self.position_status = 0

    def next(self):
        # print(f'Right now the zscore is {self.transform.zscore[0]}, and the position is {self.position_status}')
        
        # Calculate the ratio between the 2 assets
        ratio = self.data1.close[0] / self.data0.close[0]
        cash = self.broker.get_cash()

        if abs(self.zscore[0]) < self.p.CLOS_THRE and self.position_status != 0:
            print("------")
            print("close position")
            self.position_status = 0
            self.close(data=self.data0)
            self.close(data=self.data1)
    
        elif self.zscore[0] < -self.p.OPEN_THRE and self.position_status == 0:
            print("------")
            print(f"long {self.data0.alias} and short {self.data1.alias}")
            self.position_status = 1

            purchase_amount = self.broker.get_cash()/self.data0.close[0]*self.kc_f[0]

            self.sell(data=self.data1, size=purchase_amount/ratio)
            self.buy(data=self.data0, size=purchase_amount)

        elif self.zscore[0] > self.p.OPEN_THRE and self.position_status == 0:
            print("------")
            print(f"long {self.data1.alias} and short {self.data0.alias}")
            self.position_status = -1
            
            purchase_amount = self.broker.get_cash()/self.data1.close[0]*self.kc_f[0]

            self.sell(data=self.data0, size=purchase_amount*ratio)
            self.buy(data=self.data1, size=purchase_amount)

    def stop(self):
        print('==================================================')
        print('Starting Value - %.2f' % self.broker.startingcash)
        print('Ending   Value - %.2f' % self.broker.getvalue())
        print('==================================================')

# Execute the Strategy

Load the data

In [212]:
datafeeds_0 = []
datafeeds_1 = []

for idx, freq in enumerate(freqs):
    datafeeds_0.append(
        bt.feeds.PandasData(
            dataname=dfs[idx][dfs[idx]['tic']=='ETHUSDT'],
            datetime='datetime',
            open='open',
            high='high',
            low='low',
            close='close',
            volume='volume',
            openinterest=None
        )
    )
    datafeeds_1.append(
        bt.feeds.PandasData(
            dataname=dfs[idx][dfs[idx]['tic']=='BTCUSDT'],
            datetime='datetime',
            open='open',
            high='high',
            low='low',
            close='close',
            volume='volume',
            openinterest=None
        )
    )
    print(freq)

1d


The main strategy engine

In [220]:
datafeed0 = datafeeds_0[0]
datafeed1 = datafeeds_1[0]
datafeeds = [datafeed0, datafeed1]
param = {'OPEN_THRE':1, 'CLOS_THRE':0.1, 'period':30}

def cerebro_run(datafeeds, param):
    # Create a Cerebro instance and add the data feed
    cerebro = bt.Cerebro()
    cerebro.adddata(datafeeds[0], name='eth')
    cerebro.adddata(datafeeds[1], name='btc')

    # Set up other parameters for backtest
    cerebro.broker.set_cash(10000)  # Set initial capital
    
    comminfo = PairTradingCommInfo(commission=0.002, margin=1000, mult=10)
    cerebro.broker.addcommissioninfo(comminfo)

    # cerebro.addanalyzer(bt.analyzers.TimeReturn, _name='timereturns')
    cerebro.addanalyzer(bt.analyzers.Returns, _name='Returns')
    # cerebro.addsizer(KellyCriterionSizer)

    cerebro.addstrategy(PairTrading, **param)
    strats = cerebro.run()
    return strats

# cerebro_run(datafeeds, param)

# Grid Search the Strategy
Define scoring and param_grid

In [221]:
# param_grid = {
#     'OPEN_THRE':np.arange(1, 3, 1), 
#     'CLOS_THRE':np.arange(0.2, 1.0, 0.2), 
#     'period': np.arange(30, 60, 10)
# }

param_grid = {
    'OPEN_THRE': [2], 
    'CLOS_THRE': [0.5], 
    'period': [30]
}

def scoring(strats):
    score = strats[0].analyzers.Returns.get_analysis()['rtot']
    return score

Grid Searching

In [222]:
gridsearch(cerebro_run, param_grid, scoring, datafeeds)

------
long () and short ()
Sell eth @ price: 1673.73 for Qty: -5.974679309088085
Buy btc @ price: 24998.78 for Qty: 0.4000195209526225
------
close position
Buy eth @ price: 1773.88 for Qty: 5.974679309088085
Sell btc @ price: 27968.050000000003 for Qty: -0.4000195209526225
------
long () and short ()
Sell eth @ price: 1889.8600000000001 for Qty: -3.046203668143527
Buy btc @ price: 30200.43 for Qty: 0.19062312591274314
------
close position
Buy eth @ price: 1917.4 for Qty: 3.046203668143527
Sell btc @ price: 29888.07 for Qty: -0.19062312591274314
------
long () and short ()
Sell btc @ price: 28243.650000000005 for Qty: -0.17655300988593672
Buy eth @ price: 1942.98 for Qty: 2.566419323752657
------
close position
Sell eth @ price: 1874.1699999999998 for Qty: -2.566419323752657
Buy btc @ price: 27816.849999999995 for Qty: 0.17655300988593672
------
long () and short ()
Sell eth @ price: 1751.52 for Qty: -7.580900273209786
Buy btc @ price: 25841.22 for Qty: 0.5138371715386059
------
clos

(0.7090948115646988, {'OPEN_THRE': 2, 'CLOS_THRE': 0.5, 'period': 30})